# battery, glass, plastic, metal cutmix하기

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
import os
import json
import copy
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import matplotlib.patches as patches
import albumentations as A
import cv2
from tqdm import tqdm

In [2]:
coco = COCO('/opt/ml/detection/dataset/train_revision.json')

img_id = coco.getImgIds()
img_info = coco.loadImgs(img_id)
fnames = [info['file_name'] for info in img_info]

with open(os.path.join('/opt/ml/detection/dataset/train_revision.json'), 'r') as read_file:
    json_data = json.load(read_file)
new_json_data = copy.deepcopy(json_data)
print(json_data.keys())
print(len(json_data['images']), json_data['images'][-1])
print(len(json_data['annotations']), json_data['annotations'][-1])

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])
4883 {'width': 1024, 'height': 1024, 'file_name': 'train/4882.jpg', 'license': 0, 'flickr_url': None, 'coco_url': None, 'date_captured': '2020-12-23 16:20:30', 'id': 4882}
23102 {'image_id': 4882, 'category_id': 1, 'area': 149633.22, 'bbox': [145.4, 295.4, 420.2, 356.1], 'iscrowd': 0, 'id': 23143}


In [3]:
annotations = copy.deepcopy(json_data['annotations'])

In [4]:
cat_ids = [3, 4, 5, 8] # Metal, Glass, Plastic, Battery
annotation_id_per_cls = [[] for _ in range(10)]
for ann in json_data['annotations']:
    if ann['category_id'] in cat_ids:
        annotation_id_per_cls[ann['category_id']].append(ann['id'])


In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

In [6]:
annotation_id_per_cls[8].extend(annotation_id_per_cls[8])
for i in range(10):
    num = len(annotation_id_per_cls[i])
    print(f"{classes[i]} : {num}")

General trash : 0
Paper : 0
Paper pack : 0
Metal : 930
Glass : 977
Plastic : 2948
Styrofoam : 0
Plastic bag : 0
Battery : 320
Clothing : 0


In [7]:
metal_idx = np.random.choice(annotation_id_per_cls[3], len(annotation_id_per_cls[8]), replace=False)
glass_idx = np.random.choice(annotation_id_per_cls[4], len(annotation_id_per_cls[8]), replace=False)
battery_idx = np.random.choice(annotation_id_per_cls[8], len(annotation_id_per_cls[8]), replace=False)

In [8]:
new_img_id = 4883
new_ann_img_id = len(new_json_data['images'])
new_ann_id = len(new_json_data['annotations'])
print(new_ann_img_id)

4883


In [9]:
for m, g, b in tqdm(zip(metal_idx, glass_idx, battery_idx), total=len(battery_idx)):
    m_ann = annotations[m]
    g_ann = annotations[g]
    b_ann = annotations[b]
    
    category_id = [3, 4, 8]

    boxes_mgb = []
    m_bboxes = annotations[m]['bbox']
    g_bboxes = annotations[g]['bbox']
    b_bboxes = annotations[b]['bbox']
    
    boxes_mgb.append(m_bboxes)
    boxes_mgb.append(g_bboxes)
    boxes_mgb.append(b_bboxes)

    imgs_mgb = []
    imgs_mgb.append(Image.open(os.path.join('/opt/ml/detection/dataset', coco.loadImgs(m_ann['image_id'])[0]['file_name'])).crop((m_bboxes[0], m_bboxes[1], m_bboxes[2]+m_bboxes[0], m_bboxes[3]+m_bboxes[1])))
    imgs_mgb.append(Image.open(os.path.join('/opt/ml/detection/dataset', coco.loadImgs(g_ann['image_id'])[0]['file_name'])).crop((g_bboxes[0], g_bboxes[1], g_bboxes[2]+g_bboxes[0], g_bboxes[3]+g_bboxes[1])))
    imgs_mgb.append(Image.open(os.path.join('/opt/ml/detection/dataset', coco.loadImgs(b_ann['image_id'])[0]['file_name'])).crop((b_bboxes[0], b_bboxes[1], b_bboxes[2]+b_bboxes[0], b_bboxes[3]+b_bboxes[1])))

    new_boxes_mgb = []
    for idx, img in enumerate(imgs_mgb):
        random_flip = np.random.randint(0, 2)
        img_h, img_w = np.array(img).shape[:2]
        fixed_size = 400
        if img_h >= img_w:
            ratio = fixed_size / img_h
            img_h = fixed_size
            img_w = int(img_w*ratio)+1 if int(img_w*ratio)%2!=0 else int(img_w*ratio)
            
        else:
            ratio = fixed_size / img_w
            img_w = fixed_size
            img_h = int(img_h*ratio)+1 if int(img_h*ratio)%2!=0 else int(img_h*ratio)
        tb_pad = (512-img_h)//2
        lr_pad = (512-img_w)//2
        
        img = img.resize((img_w,img_h))
        img_np = np.asarray(img)
        img_np = np.pad(img_np, (((tb_pad, tb_pad),(lr_pad, lr_pad), (0,0))))
        
        # print(random_flip)
        imgs_mgb[idx] = np.flip(img_np, axis=random_flip)

        new_boxes_mgb.append([lr_pad, tb_pad, img_w, img_h])

    # fig, ax = plt.subplots(1, 1, dpi=150)
    
    imgs_mgb.append(np.zeros_like(imgs_mgb[0]))
    new_boxes_mgb.append([])
    order = np.random.choice(np.arange(4), 4, replace=False)
    
    concatenated_img = np.concatenate( (np.concatenate((imgs_mgb[order[0]], imgs_mgb[order[1]]), axis=1), np.concatenate((imgs_mgb[order[2]], imgs_mgb[order[3]]), axis=1)) , axis=0)
    

    for i in range(4):
        if len(new_boxes_mgb[order[i]])==0:
            continue
        x = i%2
        y = i//2
        new_boxes_mgb[order[i]][0],new_boxes_mgb[order[i]][1]  = new_boxes_mgb[order[i]][0]+x*512, new_boxes_mgb[order[i]][1]+y*512 

    fname = os.path.join('/opt/ml/detection/dataset/train', str(new_img_id)+'.jpg')
    
    PIL_img = Image.fromarray(concatenated_img)
    PIL_img.save(fname)
    
    temp = copy.deepcopy(new_json_data['images'][0])
    temp['file_name'] = 'train' + '/' + str(new_img_id) + '.jpg'
    temp['id'] = new_ann_img_id
    new_json_data['images'].append(temp)

    for idx, boxes in enumerate(new_boxes_mgb):
        temp = copy.deepcopy(new_json_data['annotations'][0])
        if(len(boxes)==0):
            continue
        temp['image_id'] = new_ann_img_id
        temp['category_id'] = category_id[idx]
        temp['area'] = boxes[2]*boxes[3]
        temp['bbox'] = boxes
        temp['id'] = new_ann_id
        new_json_data['annotations'].append(temp)
        new_ann_id += 1
    new_ann_img_id += 1
    new_img_id += 1
    


100%|██████████| 320/320 [00:30<00:00, 10.59it/s]


In [10]:
with open('/opt/ml/detection/dataset/negative_mining.json', 'w', encoding='utf-8') as make_file:
        json.dump(new_json_data, make_file, indent='\t')

In [115]:
# dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])
# 4883 {'width': 1024, 'height': 1024, 'file_name': 'train/4882.jpg', 'license': 0, 'flickr_url': None, 'coco_url': None, 'date_captured': '2020-12-23 16:20:30', 'id': 4882}
# 23144 {'image_id': 4882, 'category_id': 1, 'area': 149633.22, 'bbox': [145.4, 295.4, 420.2, 356.1], 'iscrowd': 0, 'id': 23143}